## 맛있는 절약 : 평가용 dataset 생성
1. find_keyIngredients_tasty : (사전 생성) 레시피의 핵심 맛과 핵심 재료를 판단
2. generate_food_group_radio : (사전 생성) 레시피의 식품군을 판별
3. fridge_recipe_transform : (기능) 냉장고 재료 대체 
4. simple_recipe_transform : (기능) 레시피 간소화 
5. balance_nutrition : (기능) 영양 맞춤형 레시피 

## 일반적인 평가용 dataset 생성 case
- 검색한 것이 질문과 관련이 있는가?
- 답변이 질문과 관련이 있는가?
- 답변을 검색한 문서 안에서 답변한 것인가?(할루시네이션 check)

## 한계
검색한 결과에 대한 정확한 기준 데이터(Ground truth) 구축이 사실상 어려움
따라서, 검색한 결과에 대한 Ground truth가 존재한다면 모두 데이터셋으로 활용하고, 아니라면 질문과 답변만으로 데이터셋을 구축하여 활용함
⇒ 우리는 레시피 생성시에 RAG가 없기 때문에 질문과 답변만으로 데이터셋을 구축해서 활용함

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [4]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("맛있는 절약-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
맛있는 절약-Evaluations


## 레시피 dataset 100개 가져오기

In [3]:
import os
# import sys

# # 현재 노트북 파일의 디렉토리 (실행 중인 디렉토리)
# notebook_dir = os.getcwd()

# # 상위 폴더
# parent_dir = os.path.dirname(notebook_dir)
# sys.path.append(os.path.abspath(parent_dir))

from db import MongoDB  # src/db.py에서 MongoDB 클래스를 가져옴
import pandas as pd

def get_recipes_and_save_to_csv():
    """
    MongoDB에서 100개의 레시피를 가져와 CSV로 저장
    """
    with MongoDB() as mongo_db:
        try:
            collection_name = "recipe"  # MongoDB 컬렉션 이름
            projection = {
                "title": 1, "type_key": 1, "method_key": 1,
                "servings": 1, "cooking_time": 1,
                "ingredients": 1, "cooking_steps": 1,
                "difficulty": 1, "tips": 1,
            }
            recipes = mongo_db.find_many(collection_name, {}, projection, limit=100)

            # 데이터프레임으로 변환
            recipes_list = [dict(recipe) for recipe in recipes]
            for recipe in recipes_list:
                recipe['_id'] = str(recipe['_id'])  # ObjectId를 문자열로 변환

            df = pd.DataFrame(recipes_list)
            
            # CSV 파일 저장 경로
            csv_file_path = os.path.join(os.getcwd(), "recipeinfo_example_dataset.csv")
            df.to_csv(csv_file_path, index=False)
            print(f"레시피 데이터를 {csv_file_path}에 저장했습니다.")
        except Exception as e:
            print(f"get_recipes_and_save_to_csv() 오류 발생: {e}")

# 실행
get_recipes_and_save_to_csv()

find_many 호출됨 - 컬렉션: recipe, limit: 100
레시피 데이터를 /Users/minji/workspace/ts-ai/src/recipeinfo_example_dataset.csv에 저장했습니다.


In [3]:
import pandas as pd
recipeinfo_example_dataset = 'recipeinfo_example_dataset.csv'
df_recipeinfo_example_dataset = pd.read_csv(recipeinfo_example_dataset)
df_recipeinfo_example_dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   _id            100 non-null    object 
 1   title          100 non-null    object 
 2   type_key       100 non-null    object 
 3   method_key     100 non-null    object 
 4   servings       100 non-null    int64  
 5   cooking_time   0 non-null      float64
 6   difficulty     0 non-null      float64
 7   ingredients    100 non-null    object 
 8   cooking_steps  100 non-null    object 
 9   tips           100 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 7.9+ KB


In [3]:
import pandas as pd
userinfo_example_dataset = 'userinfo_example_dataset.csv'
df_userinfo_example_dataset = pd.read_csv(userinfo_example_dataset)
df_userinfo_example_dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   user_allergy_ingredients  10 non-null     object
 1   user_dislike_ingredients  10 non-null     object
 2   user_spicy_level          10 non-null     object
 3   user_cooking_level        10 non-null     object
 4   user_owned_ingredients    10 non-null     object
 5   user_basic_seasoning      10 non-null     object
 6   must_use_ingredients      10 non-null     object
dtypes: object(7)
memory usage: 692.0+ bytes


In [5]:
import pandas as pd

# CSV 파일 경로
recipeinfo_file = 'recipeinfo_example_dataset.csv'
userinfo_file = 'userinfo_example_dataset.csv'

# CSV 파일 읽기
df_recipeinfo = pd.read_csv(recipeinfo_file)
df_userinfo = pd.read_csv(userinfo_file)

# Recipe와 User 정보를 결합하여 JSON 형식으로 변환
def create_combined_dataset(recipeinfo_df, userinfo_df):
    combined_data = []
    for index, user_row in userinfo_df.iterrows():
        for _, recipe_row in recipeinfo_df.iterrows():
            combined_entry = {
                "user_info": {
                    "user_allergy_ingredients": eval(user_row["user_allergy_ingredients"]),
                    "user_dislike_ingredients": eval(user_row["user_dislike_ingredients"]),
                    "user_spicy_level": user_row["user_spicy_level"],
                    "user_cooking_level": user_row["user_cooking_level"],
                    "user_owned_ingredients": eval(user_row["user_owned_ingredients"]),
                    "user_basic_seasoning": eval(user_row["user_basic_seasoning"]),
                    "must_use_ingredients": eval(user_row["must_use_ingredients"]),
                },
                "recipe_info": {
                    "_id": recipe_row["_id"],
                    "title": recipe_row["title"],
                    "type_key": recipe_row["type_key"],
                    "method_key": recipe_row["method_key"],
                    "servings": recipe_row["servings"],
                    "cooking_time": recipe_row["cooking_time"] if not pd.isna(recipe_row["cooking_time"]) else "",
                    "difficulty": recipe_row["difficulty"] if not pd.isna(recipe_row["difficulty"]) else "",
                    "ingredients": eval(recipe_row["ingredients"]),
                    "cooking_steps": eval(recipe_row["cooking_steps"]),
                    "tips": eval(recipe_row["tips"]),
                },
            }
            combined_data.append(combined_entry)
    return combined_data

# 데이터셋 생성
combined_dataset = create_combined_dataset(df_recipeinfo, df_userinfo)

# JSON 데이터셋 저장 (옵션)
import json
with open('combined_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(combined_dataset, f, ensure_ascii=False, indent=4)

print("데이터셋 생성 완료. 첫 번째 데이터:")
print(combined_dataset[0])


데이터셋 생성 완료. 첫 번째 데이터:
{'user_info': {'user_allergy_ingredients': ['땅콩'], 'user_dislike_ingredients': ['양파'], 'user_spicy_level': '3단계', 'user_cooking_level': '초급', 'user_owned_ingredients': ['김치', '계란'], 'user_basic_seasoning': ['소금', '후추'], 'must_use_ingredients': ['김치']}, 'recipe_info': {'_id': '673c36d28fad4e3c931146b6', 'title': '새우 두부 계란찜', 'type_key': '반찬', 'method_key': '찜', 'servings': 1, 'cooking_time': '', 'difficulty': '', 'ingredients': ['연두부 75g(3/4모)', '칵테일새우 20g(5마리)', '달걀 30g(1/2개)', '생크림 13g(1큰술)', '설탕 5g(1작은술)', '무염버터 5g(1작은술)', '시금치 10g(3줄기)'], 'cooking_steps': ['1. 손질된 새우를 끓는 물에 데쳐 건진다.', '2. 연두부, 달걀, 생크림, 설탕에 녹인 무염버터를 믹서에 넣고 간 뒤 새우(1)를 함께 섞어 그릇에 담는다.', '3. 시금치를 잘게 다져 혼합물 그릇(2)에 뿌리고 찜기에 넣고 중간 불에서 10분 정도 찐다.'], 'tips': ['나트륨의 배출을 도와주는 것으로 알려진 칼륨이 풍부한 시금치와 소금, 간장 등의 양념 대신 새우에 들어있는 간으로 맛을 내요.']}}


In [6]:
len(combined_dataset)

1000

In [2]:
import pandas as pd

# JSON 파일 경로
file_path = 'combined_dataset.json'

# JSON 파일을 DataFrame으로 불러오기
df = pd.read_json(file_path)

df


,user_info,recipe_info
0,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146b6', 'title': '..."
1,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146b7', 'title': '..."
2,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146b8', 'title': '..."
3,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146b9', 'title': '..."
4,"{'user_allergy_ingredients': ['땅콩'], 'user_dis...","{'_id': '673c36d28fad4e3c931146ba', 'title': '..."
...,...,...
995,"{'user_allergy_ingredients': ['참깨'], 'user_dis...","{'_id': '673c36d28fad4e3c93114715', 'title': '..."
996,"{'user_allergy_ingredients': ['참깨'], 'user_dis...","{'_id': '673c36d28fad4e3c93114716', 'title': '..."
997,"{'user_allergy_ingredients': ['참깨'], 'user_dis...","{'_id': '673c36d28fad4e3c93114717', 'title': '..."
998,"{'user_allergy_ingredients': ['참깨'], 'user_dis...","{'_id': '673c36d28fad4e3c93114718', 'title': '..."
